## 공공 데이터 포털 API 사용하기 ##

In [ ]:
# 필요한 모듈 import
import os
import sys
import urllib.request
import requests
import datetime
import time
import json
import pandas as pd

In [ ]:
# 공공데이터 포털에서 승인 받은 API encoding 키

#SERVICE KEY IS NOT REGISTERED ERROR 
#factor1: 아직 서비스 키가 승인되지 않음
#factor2: 서비스 키 디코딩 필요
ServiceKey = "G5JvLoZaFaHnPmKI4uI2Gi1ku2esnL9Py6vhG56H5SSRlnJEe8JwPc2RBcHvcuBj2E9ynnIk7BJSwoR1e4W0YA%3D%3D"

In [ ]:
print(ServiceKey)

G5JvLoZaFaHnPmKI4uI2Gi1ku2esnL9Py6vhG56H5SSRlnJEe8JwPc2RBcHvcuBj2E9ynnIk7BJSwoR1e4W0YA==


In [ ]:
#[CODE 1] url 접속 요청, url 접속에 따른 request반환
def getRequestUrl(url):    
    req = urllib.request.Request(url)     #url 요청 생성
    try: 
        response = urllib.request.urlopen(req)  #url open하여 응답에 대한 변수 받음

    #정상적인 승인 코드 -> 접속 가능
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now()) # 출력 성공 및 시간을 print
            return response.read().decode('utf-8') #json 파일을 decoding(실질적으로는 string type으로 되어있다)
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [ ]:
#[CODE 2]
def getTourismStatsItem(yyyymm, national_code, ed_cd):    
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    parameters = "?_type=json&serviceKey=" + ServiceKey   #인증키
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD=" + national_code
    parameters += "&ED_CD=" + ed_cd
    url = service_url + parameters
    
    retData = getRequestUrl(url)   #[CODE 1]에서 작성한 함수를 불러옴, url에 접속하여 data를 받음
    
    if (retData == None):
        return None
    else:
         return json.loads(retData)

In [ ]:
#[CODE 3] 응답받은 data들을 다시 formating
#매개변수 -> nat_cd: 데이터를 수집할 국가 코드, ed_cd: 입국/출국 코드('E' 또는 'D')

def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    natName=''
    dataEND = "{0}{1:0>2}".format(str(nEndYear), str(12)) #데이터 끝 초기화
    isDataEnd = 0 #데이터 끝 확인용 flag 초기화
    ed=''
    
    for year in range(nStartYear, nEndYear+1):        #원하는 연도
        for month in range(1, 13):     #그에 해당하는 월 >> 연월마다 돌아감
            if(isDataEnd == 1): break #데이터 끝 flag 설정되어있으면 작업 중지.

            #format문법 -> 단순하게 말하면 월을 03형식으로 작성하겠다는 뜻
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))            

            #[CODE 2] 요청한 데이터를 응답 받는 함수. 따라서 응답 받은 데이터를 사용
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd) 
            
            if (jsonData['response']['header']['resultMsg'] == 'OK'):               
                # 입력된 범위까지 수집하지 않았지만, 더이상 제공되는 데이터가 없는 마지막 항목인 경우 -------------------
                if jsonData['response']['body']['items'] == '': 
                    isDataEnd = 1 #데이터 끝 flag 설정
                    dataEND = "{0}{1:0>2}".format(str(year), str(month-1))
                    print("데이터 없음.... \n제공되는 통계 데이터는 %s년 %s월까지입니다." %(str(year), str(month-1)))                    
                    break                
                #jsonData를 출력하여 확인......................................................
                print (json.dumps(jsonData, indent=4, sort_keys=True, ensure_ascii=False))          
                natName = jsonData['response']['body']['items']['item']['natKorNm']
                natName = natName.replace(' ', '')
                num = jsonData['response']['body']['items']['item']['num']
                ed = jsonData['response']['body']['items']['item']['ed']
                print('[ %s_%s : %s ]' %(natName, yyyymm, num))
                print('----------------------------------------------------------------------')                
                jsonResult.append({'nat_name': natName, 'nat_cd': nat_cd,'yyyymm': yyyymm, 'visit_cnt': num})
                result.append([natName, nat_cd, yyyymm, num])              
    return (jsonResult, result, natName, ed, dataEND)

In [ ]:
#[CODE 0]
#결과를 담은 result data가 지역변수로 설정되어있기 때문에 data를 담기 위한 전역변수 설정
dataframe = pd.DataFrame()

def main():
    jsonResult = []
    result = []
    natName=''
    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : ')
    nStartYear =int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E"     #E : 방한외래관광객, D : 해외 출국
    
    jsonResult, result, natName, ed, dataEND =getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear) #[CODE 3]

    if (natName=='') : #URL 요청은 성공하였지만, 데이터 제공이 안된 경우
        print('데이터가 전달되지 않았습니다. 공공데이터포털의 서비스 상태를 확인하기 바랍니다.')
    else:
        save_dir = "/content/sample_data" #저장할 경로: 코랩 directory
        csv_name = '/%s_%s_%d_%s.csv' % (natName, ed, nStartYear, dataEND)
        json_name = '/%s_%s_%d_%s.json' % (natName, ed, nStartYear, dataEND)

        #파일저장 1 : json 파일      
        #natName_ed_nStartYear_dataEND.json 이름으로 file 생성 
        with open(save_dir + json_name, 'w', encoding='utf8') as outfile:
            jsonFile  = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
            outfile.write(jsonFile)

        #파일저장 2 : csv 파일   
        #natName_ed_nStartYear_dataEND.csv 이름으로 file 생성
        columns = ["입국자국가", "국가코드", "입국연월", "입국자 수"]
        dataframe = pd.DataFrame(result, columns = columns)
        dataframe.to_csv(save_dir + csv_name, index=False, encoding='cp949')

In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd

#[CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None


#[CODE 2]
def getTourismStatsItem(yyyymm, national_code, ed_cd):    
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    parameters = "?_type=json&serviceKey=" + ServiceKey   #인증키
    parameters += "&YM=" + yyyymm
    parameters += "&NAT_CD=" + national_code
    parameters += "&ED_CD=" + ed_cd
    url = service_url + parameters
    
    retData = getRequestUrl(url)   #[CODE 1]
    
    if (retData == None):
        return None
    else:
         return json.loads(retData)

#[CODE 3]
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    natName=''
    dataEND = "{0}{1:0>2}".format(str(nEndYear), str(12)) #데이터 끝 초기화
    isDataEnd = 0 #데이터 끝 확인용 flag 초기화
    ed=''
    
    for year in range(nStartYear, nEndYear+1):        
        for month in range(1, 13):
            if(isDataEnd == 1): break #데이터 끝 flag 설정되어있으면 작업 중지.
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))            
            jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd) #[CODE 2]
            
            if (jsonData['response']['header']['resultMsg'] == 'OK'):               
                # 입력된 범위까지 수집하지 않았지만, 더이상 제공되는 데이터가 없는 마지막 항목인 경우 -------------------
                if jsonData['response']['body']['items'] == '': 
                    isDataEnd = 1 #데이터 끝 flag 설정
                    dataEND = "{0}{1:0>2}".format(str(year), str(month-1))
                    print("데이터 없음.... \n제공되는 통계 데이터는 %s년 %s월까지입니다." %(str(year), str(month-1)))                    
                    break                
                #jsonData를 출력하여 확인......................................................
                print (json.dumps(jsonData, indent=4, sort_keys=True, ensure_ascii=False))          
                natName = jsonData['response']['body']['items']['item']['natKorNm']
                natName = natName.replace(' ', '')
                num = jsonData['response']['body']['items']['item']['num']
                ed = jsonData['response']['body']['items']['item']['ed']
                print('[ %s_%s : %s ]' %(natName, yyyymm, num))
                print('----------------------------------------------------------------------')                
                jsonResult.append({'nat_name': natName, 'nat_cd': nat_cd,'yyyymm': yyyymm, 'visit_cnt': num})
                result.append([natName, nat_cd, yyyymm, num])              
    return (jsonResult, result, natName, ed, dataEND)

#[CODE 0]
def main():
    jsonResult = []
    result = []
    natName=''
    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : ')
    nStartYear =int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E"     #E : 방한외래관광객, D : 해외 출국
    
    jsonResult, result, natName, ed, dataEND =getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear) #[CODE 3]

    if (natName=='') : #URL 요청은 성공하였지만, 데이터 제공이 안된 경우
        print('데이터가 전달되지 않았습니다. 공공데이터포털의 서비스 상태를 확인하기 바랍니다.')

        #파일저장 1 : json 파일      
    else: 
        with open('./%s_%s_%d_%s.json' % (natName, ed, nStartYear, dataEND), 'w', encoding='utf8') as outfile:
            jsonFile  = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
            outfile.write(jsonFile)
        #파일저장 2 : csv 파일   
        columns = ["입국자국가", "국가코드", "입국연월", "입국자 수"]
        result_df = pd.DataFrame(result, columns = columns)
        result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear, dataEND),
                    index=False, encoding='cp949')
        print(result)

In [ ]:
if __name__ == '__main__':
  main()

<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>
국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : 112
데이터를 몇 년부터 수집할까요? : 2019
데이터를 몇 년까지 수집할까요? : 2019
[2023-04-12 16:48:35.123387] Url Request Success
[2023-04-12 16:48:35.449095] Url Request Success
[2023-04-12 16:48:35.765243] Url Request Success
[2023-04-12 16:48:36.088262] Url Request Success
[2023-04-12 16:48:36.402016] Url Request Success
[2023-04-12 16:48:36.725178] Url Request Success
[2023-04-12 16:48:37.045411] Url Request Success
[2023-04-12 16:48:37.364301] Url Request Success
[2023-04-12 16:48:37.680306] Url Request Success
[2023-04-12 16:48:38.004702] Url Request Success
[2023-04-12 16:48:38.331909] Url Request Success
[2023-04-12 16:48:38.649413] Url Request Success
데이터가 전달되지 않았습니다. 공공데이터포털의 서비스 상태를 확인하기 바랍니다.


In [ ]:
#data check
dataframe.head()

""
